# **ETL to InfoCorridasDia**

## Dependencies

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("LeituraParquet").getOrCreate()

In [3]:
from pyspark.sql.functions import substring, count, col, when, desc, min, max, sum, round, to_date, date_format

In [4]:
# File path
parquet_path = "/Users/andrebezerra/Desktop/Dev/DesafiosCodeElevate/diario_de_bordo/transportes.parquet"

In [7]:
# Read parquet
df_spark = spark.read.parquet(parquet_path)

In [6]:
# Remover a parte da hora da DATA_INICIO
df_spark = df_spark.withColumn("DATA_INICIO", substring("DATA_INICIO", 1, 10))

# Convertendo para formato yyyy-MM-dd
df_spark = df_spark.withColumn("DATA_INICIO", to_date(col("DATA_INICIO"), "MM-dd-yyyy"))

# Agregar os dados
df_resultado = df_spark.groupBy("DATA_INICIO").agg(
    count(col("LOCAL_INICIO")).alias("QT_CORR"),        # total travels
    count(when(col("CATEGORIA") == "Negocio", True)).alias("QT_CORR_NEG"),      # Count travels like 'Negocio'
    count(when(col("CATEGORIA") == "Pessoal", True)).alias("QT_CORR_PESS"),     # Count travels like 'Pessoal'
    max(col("DISTANCIA")).alias("VL_MAX_DIST"),     # longest distance of the day
    min(col("DISTANCIA")).alias("VL_MIN_DIST"),     # shortest distance of the day
    round(sum(col("DISTANCIA")) / count(col("DATA_INICIO")), 2).alias("VL_AVG_DIST"),       # average total distance
    count(when(col("PROPOSITO").isin("Reunião"), True)).alias("QT_CORR_REUNI"),     # Travels for the purpose of "Reunião".
    count(when((col("PROPOSITO").isNotNull()) & (~col("PROPOSITO").isin("Reunião")), True)).alias("QT_CORR_NAO_REUNI")      # Travels with a stated purpose other than "Reunião".  
).orderBy(col("DATA_INICIO").desc())
df_resultado.show(15)

+-----------+-------+-----------+------------+-----------+-----------+-----------+-------------+-----------------+
|DATA_INICIO|QT_CORR|QT_CORR_NEG|QT_CORR_PESS|VL_MAX_DIST|VL_MIN_DIST|VL_AVG_DIST|QT_CORR_REUNI|QT_CORR_NAO_REUNI|
+-----------+-------+-----------+------------+-----------+-----------+-----------+-------------+-----------------+
| 2016-12-31|      5|          5|           0|        482|          7|      150.8|            2|                3|
| 2016-12-30|      5|          5|           0|         46|          8|       31.4|            1|                4|
| 2016-12-29|     13|         13|           0|        129|          3|      55.54|            2|               10|
| 2016-12-28|      7|          7|           0|        104|          2|       61.0|            0|                7|
| 2016-12-27|      6|          6|           0|         79|          5|       37.5|            1|                5|
| 2016-12-26|      5|          5|           0|         79|         32|       57.